In [32]:
from io import StringIO  # extract
import requests 
from bs4 import BeautifulSoup
import pandas as pd # transformation
import sqlite3 #load
from datetime import datetime
from icecream import ic

In [22]:
pip install icecream

In [23]:
from icecream import ic

In [24]:
for i in range(1,5):
    a = 200
    b = 250
    c = 300
    ans = a*b/c
    a-= 10
    b+= 20
    print(a)
    print(b)
    print(ans)

190
270
166.66666666666666
190
270
166.66666666666666
190
270
166.66666666666666
190
270
166.66666666666666


In [25]:
for i in range(1,5):
    ans  = ic(a)*(b)/(c)
    a-=10
    b+=20


ic| Error: Failed to access the underlying source code for analysis. Was ic() invoked in a REPL (e.g. from the command line), a frozen application (e.g. packaged with PyInstaller), or did the underlying source code change during execution?
ic| Error: Failed to access the underlying source code for analysis. Was ic() invoked in a REPL (e.g. from the command line), a frozen application (e.g. packaged with PyInstaller), or did the underlying source code change during execution?
ic| Error: Failed to access the underlying source code for analysis. Was ic() invoked in a REPL (e.g. from the command line), a frozen application (e.g. packaged with PyInstaller), or did the underlying source code change during execution?
ic| Error: Failed to access the underlying source code for analysis. Was ic() invoked in a REPL (e.g. from the command line), a frozen application (e.g. packaged with PyInstaller), or did the underlying source code change during execution?


# Step 0

In [52]:
# step 0 
def log_progress(message):
    with open ('./code_;og.txt', 'a') as f:
        f.write(f" {datetime.now()}: {message}.\n")

# step1 

In [53]:
def extract(url, table_att):
    soup = requests.get(url).text
    web = BeautifulSoup(soup, 'html.parser')
    table = web.find('span', string=table_att).find_next('table')
    df = pd.read_html(StringIO(str(table)))[0]

    log_progress('Data Extracdted Succesfully from the wikipedia')

    return df

# Step 2

In [54]:
def transformation(df, csv_path):
    exchange_rate = pd.read_csv(csv_path, index_col = 0).to_dict()['Rate']

    df['MC_GBP_Billion'] = round(df['Market cap (US$ billion)']*exchange_rate['GBP'],2)
    df['MC_EUR_Billion'] = round(df['Market cap (US$ billion)']*exchange_rate['EUR'],2)
    df['MC_INR_Billion'] = round(df['Market cap (US$ billion)']*exchange_rate['INR'],2)

    print(df['MC_EUR_Billion'][4])

    log_progress('Data Transformation done now, Step 2 Complete.')
    return df
    

# Step 3

In [55]:
def load_to_csv(df, out_path):
    df.to_csv(out_path)

    log_progress('data loaded into the csv file')

In [56]:
# step 3
def load_to_db(df, sql_conn, tn):
    df.to_sql(tn, sql_conn, if_exists='replace', index=False)

    log_progress('Data loaded into the SQLITE database')

In [57]:
if __name__ == '__main__':
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output_csv_path = './Largest_banks_data.csv'
    database_name = 'Banks.db'
    table_name = 'Largest_banks'
    
    log_progress('Preliminaries complete. Initiating ETL process')

    print(extract(url, 'By market capitalization'))

   Rank                                Bank name  Market cap (US$ billion)
0     1                           JPMorgan Chase                    432.92
1     2                          Bank of America                    231.52
2     3  Industrial and Commercial Bank of China                    194.56
3     4               Agricultural Bank of China                    160.68
4     5                                HDFC Bank                    157.91
5     6                              Wells Fargo                    155.87
6     7                        HSBC Holdings PLC                    148.90
7     8                           Morgan Stanley                    140.83
8     9                  China Construction Bank                    139.82
9    10                            Bank of China                    136.81


In [58]:
if __name__ == '__main__':
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output_csv_path = './Largest_banks_data.csv'
    database_name = 'Banks.db'
    table_name = 'Largest_banks'
    
    log_progress('Preliminaries complete. Initiating ETL process')

    df=(extract(url, 'By market capitalization'))
    log_progress('Extraction completeed')
    print(df)
    
    df = transformation(df, './input/exchange_rate.csv')
    log_progress('Data Transformed sucessfully')
    print(df)

   Rank                                Bank name  Market cap (US$ billion)
0     1                           JPMorgan Chase                    432.92
1     2                          Bank of America                    231.52
2     3  Industrial and Commercial Bank of China                    194.56
3     4               Agricultural Bank of China                    160.68
4     5                                HDFC Bank                    157.91
5     6                              Wells Fargo                    155.87
6     7                        HSBC Holdings PLC                    148.90
7     8                           Morgan Stanley                    140.83
8     9                  China Construction Bank                    139.82
9    10                            Bank of China                    136.81
146.86
   Rank                                Bank name  Market cap (US$ billion)  \
0     1                           JPMorgan Chase                    432.92   
1     2     

In [60]:
if __name__ == '__main__':
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output_csv_path = './Largest_banks_data.csv'
    database_name = 'Banks.db'
    table_name = 'Largest_banks'
    
    log_progress('Preliminaries complete. Initiating ETL process')

    df=(extract(url, 'By market capitalization'))
    log_progress('Extraction completeed')
    print(df)
    
    df = transformation(df, './input/exchange_rate.csv')
    log_progress('Data Transformed sucessfully')
    print(df)

    load_to_csv(df, output_csv_path)
    log_progress('DataFrame has beenn loaded into csv')

    with sqlite3.connect(database_name) as conn:
        load_to_db(df, conn, table_name)
    log_progress('DataFrame has been uploaded into the SQlite Database Sucessfully')

   Rank                                Bank name  Market cap (US$ billion)
0     1                           JPMorgan Chase                    432.92
1     2                          Bank of America                    231.52
2     3  Industrial and Commercial Bank of China                    194.56
3     4               Agricultural Bank of China                    160.68
4     5                                HDFC Bank                    157.91
5     6                              Wells Fargo                    155.87
6     7                        HSBC Holdings PLC                    148.90
7     8                           Morgan Stanley                    140.83
8     9                  China Construction Bank                    139.82
9    10                            Bank of China                    136.81
146.86
   Rank                                Bank name  Market cap (US$ billion)  \
0     1                           JPMorgan Chase                    432.92   
1     2     

In [ ]:
# Hi LinkedIn Community,
Sharing my project in Data Engineering - Building a Serverless ETL Application on AWS - https://lnkd.in/gaN-BS9V

A loads of thanks to David Freitag for teaching such a wonderful project-based course and all the guidance without whom this project wouldn't be successful.

Below is the LinkedIn Article where I have shared my learnings through the course and building end-to-end Serverless Data Engineering ETL Application project.
